In [1]:
import spotipy
import spotipy.oauth2
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import statistics
import matplotlib.pyplot as plt
import time

%matplotlib inline

In [2]:
"dat = pd.read_csv('removed_duplicates.csv')"

"dat = pd.read_csv('removed_duplicates.csv')"

In [3]:
"rel_subset = dat.sample(n = 2000)"

'rel_subset = dat.sample(n = 2000)'

In [4]:
"rel_subset.columns"

'rel_subset.columns'

In [5]:
"rel_subset = rel_subset.drop('Unnamed: 0', axis = 'columns')"

"rel_subset = rel_subset.drop('Unnamed: 0', axis = 'columns')"

In [6]:
"""rel_subset = rel_subset.drop_duplicates(subset = ['uri'])
print(len(rel_subset) == len(set(rel_subset['uri'])))
print(rel_subset.shape)
rel_subset.head(2)"""

"rel_subset = rel_subset.drop_duplicates(subset = ['uri'])\nprint(len(rel_subset) == len(set(rel_subset['uri'])))\nprint(rel_subset.shape)\nrel_subset.head(2)"

In [7]:
CLIENT_ID = '23888f5deee6452db5d78bd2d1091da5'
CLIENT_SECRET = 'f6ecea7a89d24b02b2c969745b88a9db'

client_credentials_manager = SpotifyClientCredentials(CLIENT_ID, CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [8]:
"genre_dict = {}"

'genre_dict = {}'

In [9]:
"""%%time
for i in range(0, len(rel_subset)):
    curr_uri = list(rel_subset['uri'])[i]
    if curr_uri not in genre_dict.keys():
        possible_genre = sp.artist(sp.track(curr_uri)['artists'][0]['uri'])['genres']
        if possible_genre == []:
            genre_dict[curr_uri] = 'None'
        else:
            genre_dict[curr_uri] = possible_genre[0]
    if i % 100 == 0:
        print(len(rel_subset) - i, 'remaining')"""

"%%time\nfor i in range(0, len(rel_subset)):\n    curr_uri = list(rel_subset['uri'])[i]\n    if curr_uri not in genre_dict.keys():\n        possible_genre = sp.artist(sp.track(curr_uri)['artists'][0]['uri'])['genres']\n        if possible_genre == []:\n            genre_dict[curr_uri] = 'None'\n        else:\n            genre_dict[curr_uri] = possible_genre[0]\n    if i % 100 == 0:\n        print(len(rel_subset) - i, 'remaining')"

In [10]:
"""rel_subset['genre'] = ''
print(len(rel_subset['genre']))"""

"rel_subset['genre'] = ''\nprint(len(rel_subset['genre']))"

In [11]:
"""def f(row):
    return(genre_dict[row])"""

'def f(row):\n    return(genre_dict[row])'

In [12]:
"rel_subset['genre'] = rel_subset['uri'].apply(f)"

"rel_subset['genre'] = rel_subset['uri'].apply(f)"

In [13]:
"rel_subset.columns"

'rel_subset.columns'

In [14]:
#rel_subset.to_csv('subset_500mb.csv')

In [15]:
%%time
dat = pd.read_csv('subset_500mb.csv')

CPU times: user 62 ms, sys: 7 ms, total: 69 ms
Wall time: 121 ms


In [16]:
#all_timbres_1 = pd.read_csv('timbres_all_0_2500.csv')

In [17]:
#all_timbres_2 = pd.read_csv('timbres_all_2500_5000.csv')

In [18]:
#all_timbres_3 = pd.read_csv('timbres_all_5000_7500.csv')

In [19]:
#all_timbres_4 = pd.read_csv('timbres_all_7500_10000.csv')

In [20]:
#all_timbres_5 = pd.read_csv('timbres_all_10000_12500.csv')

In [21]:
#all_timbres_6 = pd.read_csv('timbres_all_12500_15000.csv')

In [22]:
#all_timbres_7 = pd.read_csv('timbres_all_15000_17500.csv')

In [23]:
#all_timbres_8 = pd.read_csv('timbres_all_17500_20000.csv')

In [24]:
#all_timbres = pd.read_csv('timbres_all_unique.csv', nrows = 2)

In [25]:
timbre_all_labels = ['timbre_all_1', 'timbre_all_2', 'timbre_all_3', 'timbre_all_4', 
                 'timbre_all_5', 'timbre_all_6', 'timbre_all_7',
                 'timbre_all_8', 'timbre_all_9', 'timbre_all_10', 'timbre_all_11',
                 'timbre_all_12']
timbre_all_df = pd.DataFrame(index = range(len(dat)), columns = timbre_all_labels)
print(timbre_all_df.shape)
timbre_all_df['timbre_all_1'][0] = 1
timbre_all_df.head(2)

(1592, 12)


,timbre_all_1,timbre_all_2,timbre_all_3,timbre_all_4,timbre_all_5,timbre_all_6,timbre_all_7,timbre_all_8,timbre_all_9,timbre_all_10,timbre_all_11,timbre_all_12
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
%%time
for i in range(0, len(dat['uri'])): #iterate through all uris
    start_time = time.time()
    curr_uri = dat['uri'][i]
    curr_song = sp.audio_analysis(curr_uri)
    curr_segments = curr_song['segments'] #get into segments (time intervals). Each song diff length
    
    #init_mean_timbres = pd.Series(np.repeat(0, 12)) #initialize timbre value holder for means
    init_timbres_all = [[] for _ in range(12)] #initialize 12 lists for median (find median each list)
    
    for j in range(0, len(curr_segments)):
        curr_timbres_list = curr_segments[j]['timbre'] #grab timbres as list
        #curr_timbres_series = pd.Series(curr_timbres_list) #we have to iterate through every segment
        #print(curr_timbres)
        #init_mean_timbres += curr_timbres_series
        
        init_timbres_all[0].append(curr_timbres_list[0])
        init_timbres_all[1].append(curr_timbres_list[1])
        init_timbres_all[2].append(curr_timbres_list[2])
        init_timbres_all[3].append(curr_timbres_list[3])
        init_timbres_all[4].append(curr_timbres_list[4])
        init_timbres_all[5].append(curr_timbres_list[5])
        init_timbres_all[6].append(curr_timbres_list[6])
        init_timbres_all[7].append(curr_timbres_list[7])
        init_timbres_all[8].append(curr_timbres_list[8])
        init_timbres_all[9].append(curr_timbres_list[9])
        init_timbres_all[10].append(curr_timbres_list[10])
        init_timbres_all[11].append(curr_timbres_list[11])
    
    timbre_all_df['timbre_all_1'][i] = init_timbres_all[0]
    timbre_all_df['timbre_all_2'][i] = init_timbres_all[1]
    timbre_all_df['timbre_all_3'][i] = init_timbres_all[2]
    timbre_all_df['timbre_all_4'][i] = init_timbres_all[3]
    timbre_all_df['timbre_all_5'][i] = init_timbres_all[4]
    timbre_all_df['timbre_all_6'][i] = init_timbres_all[5]
    timbre_all_df['timbre_all_7'][i] = init_timbres_all[6]
    timbre_all_df['timbre_all_8'][i] = init_timbres_all[7]
    timbre_all_df['timbre_all_9'][i] = init_timbres_all[8]
    timbre_all_df['timbre_all_10'][i] = init_timbres_all[9]
    timbre_all_df['timbre_all_11'][i] = init_timbres_all[10]
    timbre_all_df['timbre_all_12'][i] = init_timbres_all[11]
    
    print(i, time.time() - start_time)

0 1.6434321403503418
1 1.3908967971801758
2 1.5999658107757568
3 1.4362494945526123
4 1.636199951171875
5 1.3515889644622803
6 1.5280835628509521
7 1.2093491554260254
8 1.2794742584228516
9 1.26090407371521
10 1.2997732162475586
11 1.118211030960083
12 1.2240972518920898
13 1.375986099243164
14 1.2427661418914795
15 1.3328430652618408
16 1.253587245941162
17 1.2490127086639404
18 1.1146295070648193
19 1.2289280891418457
20 1.3426196575164795
21 1.2178668975830078
22 1.2781996726989746
23 1.2547297477722168
24 1.4723896980285645
25 1.4153649806976318
26 1.5395843982696533
27 1.4573137760162354
28 1.3391697406768799
29 1.3182060718536377
30 1.3493998050689697
31 1.237560510635376
32 1.278024673461914
33 1.3269329071044922
34 1.2095580101013184
35 1.21760892868042
36 1.2152485847473145
37 1.104353904724121
38 1.2378990650177002
39 1.2748780250549316
40 1.258636713027954
41 1.4393980503082275
42 1.2630348205566406
43 1.205620288848877
44 1.2313871383666992
45 1.204787015914917
46 1.3013994

In [31]:
timbre_all_df['uri'] = dat['uri']
print(timbre_all_df.shape)
timbre_all_df.head(2)

(1592, 13)


,timbre_all_1,timbre_all_2,timbre_all_3,timbre_all_4,timbre_all_5,timbre_all_6,timbre_all_7,timbre_all_8,timbre_all_9,timbre_all_10,timbre_all_11,timbre_all_12,uri
0,"[0.0, 24.1, 23.897, 27.356, 32.848, 33.768, 36...","[171.13, -232.911, -237.644, -157.177, -88.399...","[9.469, -154.046, -145.614, -107.148, -89.892,...","[-28.48, -63.445, -84.944, -133.198, -69.415, ...","[57.491, 6.579, 15.005, -19.511, -25.059, -18....","[-50.067, -73.619, -60.84, -47.25, -49.875, -4...","[14.833, 15.024, 24.499, -6.516, -8.199, -0.71...","[5.359, 4.801, 18.949, 21.249, -0.277, -0.328,...","[-27.228, -7.805, -8.304, 40.258, 33.085, 33.6...","[0.973, 2.661, -1.462, 14.984, 18.834, 23.424,...","[-10.64, -9.55, -0.761, 12.993, -7.358, -8.305...","[-7.228, -17.296, -22.295, -10.881, -15.234, -...",spotify:track:7pueb0FHOYlKAJPXhVhwml
1,"[0.0, 20.612, 29.468, 37.065, 42.457, 50.682, ...","[171.13, 281.408, 154.885, 7.066, -17.206, 134...","[9.469, 112.323, 9.7, 65.887, 41.122, 85.356, ...","[-28.48, -106.592, 74.184, 57.284, -187.479, -...","[57.491, 32.225, 19.191, 117.453, 49.711, 85.6...","[-50.067, 59.13, 41.981, 134.899, -17.262, -12...","[14.833, -14.27, -23.128, 7.356, -26.231, -40....","[5.359, -5.607, -56.754, -3.356, 47.938, -19.8...","[-27.228, -10.957, -15.955, 30.643, -7.379, -0...","[0.973, 22.207, 15.478, 23.2, -2.171, -21.529,...","[-10.64, 41.326, -93.342, 11.952, 41.559, 5.65...","[-7.228, -12.727, 15.358, -21.875, -2.807, 4.9...",spotify:track:4pLwZjInHj3SimIyN9SnOz


In [32]:
#timbre_all_df.to_csv('timbres_subset.csv')

In [35]:
timbre_all_df = timbre_all_df.drop('uri', axis = 'columns')
print(timbre_all_df.shape)
timbre_all_df.head(2)

(1592, 12)


,timbre_all_1,timbre_all_2,timbre_all_3,timbre_all_4,timbre_all_5,timbre_all_6,timbre_all_7,timbre_all_8,timbre_all_9,timbre_all_10,timbre_all_11,timbre_all_12
0,"[0.0, 24.1, 23.897, 27.356, 32.848, 33.768, 36...","[171.13, -232.911, -237.644, -157.177, -88.399...","[9.469, -154.046, -145.614, -107.148, -89.892,...","[-28.48, -63.445, -84.944, -133.198, -69.415, ...","[57.491, 6.579, 15.005, -19.511, -25.059, -18....","[-50.067, -73.619, -60.84, -47.25, -49.875, -4...","[14.833, 15.024, 24.499, -6.516, -8.199, -0.71...","[5.359, 4.801, 18.949, 21.249, -0.277, -0.328,...","[-27.228, -7.805, -8.304, 40.258, 33.085, 33.6...","[0.973, 2.661, -1.462, 14.984, 18.834, 23.424,...","[-10.64, -9.55, -0.761, 12.993, -7.358, -8.305...","[-7.228, -17.296, -22.295, -10.881, -15.234, -..."
1,"[0.0, 20.612, 29.468, 37.065, 42.457, 50.682, ...","[171.13, 281.408, 154.885, 7.066, -17.206, 134...","[9.469, 112.323, 9.7, 65.887, 41.122, 85.356, ...","[-28.48, -106.592, 74.184, 57.284, -187.479, -...","[57.491, 32.225, 19.191, 117.453, 49.711, 85.6...","[-50.067, 59.13, 41.981, 134.899, -17.262, -12...","[14.833, -14.27, -23.128, 7.356, -26.231, -40....","[5.359, -5.607, -56.754, -3.356, 47.938, -19.8...","[-27.228, -10.957, -15.955, 30.643, -7.379, -0...","[0.973, 22.207, 15.478, 23.2, -2.171, -21.529,...","[-10.64, 41.326, -93.342, 11.952, 41.559, 5.65...","[-7.228, -12.727, 15.358, -21.875, -2.807, 4.9..."


In [39]:
dat = pd.concat([dat, timbre_all_df], axis=1)

In [40]:
dat.shape

(1592, 92)

In [ ]:
#dat.to_csv('subset_every_feature.csv')